In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from etl import ETL
from feature_creation import FeatureCreation

plt.style.use("seaborn-v0_8")

In [2]:
df_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")  # 1982 to 2018
df_climate = pd.read_parquet(
    "data/climate_data_from_1982.parquet"
)  # 1982-2014 2015-2050

In [3]:
df_yield, df_climate = ETL(df_yield, df_climate).run()

--- df_climate---
Departments/Scenario dropped because of any missing values:                                    nom_dep  scenario
date                                                
2015-01-01 12:00:00               Calvados  ssp2_4_5
2015-01-01 12:00:00            Deux_Sevres  ssp2_4_5
2015-01-01 12:00:00                Essonne  ssp2_4_5
2015-01-01 12:00:00                   Eure  ssp2_4_5
2015-01-01 12:00:00                  Rhone  ssp2_4_5
2015-01-01 12:00:00        Tarn_et_Garonne  ssp2_4_5
2015-01-01 12:00:00  Territoire_de_Belfort  ssp2_4_5
2015-01-01 12:00:00               Vaucluse  ssp2_4_5
--- df_yield ---
Departments dropped because of almost absolute absence of data:
 ['Hauts_de_Seine' 'Paris' 'Seine_SeineOise']


In [13]:
with pd.option_context("display.max_column", 999):
    display(df_hist.head())

,year,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_1,amp_daily_NSA_temp_2,amp_daily_NSA_temp_3,amp_daily_NSA_temp_4,amp_daily_NSA_temp_5,amp_daily_NSA_temp_6,amp_daily_NSA_temp_7,amp_daily_NSA_temp_8,amp_daily_NSA_temp_9,amp_daily_NSA_temp_10,amp_daily_NSA_temp_11,amp_daily_NSA_temp_12,amp_max_daily_NSA_temp_1,amp_max_daily_NSA_temp_2,amp_max_daily_NSA_temp_3,amp_max_daily_NSA_temp_4,amp_max_daily_NSA_temp_5,amp_max_daily_NSA_temp_6,amp_max_daily_NSA_temp_7,amp_max_daily_NSA_temp_8,amp_max_daily_NSA_temp_9,amp_max_daily_NSA_temp_10,amp_max_daily_NSA_temp_11,amp_max_daily_NSA_temp_12,amp_precipitation_1,amp_precipitation_2,amp_precipitation_3,amp_precipitation_4,amp_precipitation_5,amp_precipitation_6,amp_precipitation_7,amp_precipitation_8,amp_precipitation_9,amp_precipitation_10,amp_precipitation_11,amp_precipitation_12,frost_days_germination,frost_days_growth,frost_days_maturation,heat_days_germination,heat_days_growth,heat_days_maturation,rainy_days_germination,rainy_days_growth,rainy_days_maturation,germination_above,germination_below,germination_within,growth_above,growth_below,growth_within,maturation_above,maturation_below,maturation_within,total_above_consecutive_days_germination,total_above_consecutive_days_growth,total_above_consecutive_days_maturation,total_below_consecutive_days_germination,total_below_consecutive_days_growth,total_below_consecutive_days_maturation,annual_precip_below,annual_precip_within,annual_precip_above
0,1982,3.950080,16065.0,63458.0,historical,Ain,01,8.102631,12.605316,15.886414,14.585327,8.893372,9.951599,8.545471,13.266602,9.599823,14.662964,14.701843,14.508118,9.683777,14.983551,19.126007,19.172760,12.611084,12.062103,12.067383,17.436310,12.622467,16.060181,15.186584,17.565948,0.000293,0.000305,0.000566,0.000333,0.000520,0.000220,0.000355,0.000501,0.000160,0.000870,0.000488,0.000337,3.0,27.0,0.0,0.0,0.0,3.0,11.0,82.0,5.0,0.0,16.0,45.0,0.0,89.0,93.0,0.0,10.0,51.0,0.0,0.0,0.0,7.0,25.0,6.0,0.0,0.0,1.0
1,1983,2.648276,14500.0,38400.0,historical,Ain,01,9.464539,17.021851,11.039673,11.894348,10.019318,10.661621,10.034546,7.150391,10.892151,10.016907,10.962952,16.380951,9.032074,16.222992,12.593597,15.863464,13.926239,13.816620,13.089539,10.518738,14.718750,12.206482,12.406281,14.027649,0.000439,0.000397,0.000536,0.000509,0.000355,0.000392,0.000694,0.000523,0.000531,0.000217,0.000363,0.000416,5.0,48.0,0.0,0.0,0.0,0.0,22.0,62.0,14.0,0.0,17.0,44.0,0.0,102.0,80.0,0.0,24.0,37.0,0.0,0.0,0.0,9.0,46.0,12.0,0.0,0.0,1.0
2,1984,4.822580,15500.0,74750.0,historical,Ain,01,15.847443,9.180328,7.104919,14.128510,10.524353,9.929626,10.030670,5.867737,14.377869,15.163147,7.973297,12.289917,14.612091,11.839844,10.429443,20.373047,14.967377,13.784973,13.562866,10.061218,20.035645,16.957092,6.830505,12.512360,0.000419,0.000420,0.000334,0.000987,0.000682,0.000335,0.000635,0.000350,0.000394,0.000519,0.000384,0.000269,1.0,45.0,0.0,0.0,0.0,0.0,22.0,60.0,18.0,0.0,7.0,54.0,0.0,96.0,87.0,0.0,21.0,40.0,0.0,0.0,0.0,5.0,31.0,9.0,0.0,0.0,1.0
3,1985,4.196770,15500.0,65050.0,historical,Ain,01,14.638794,13.399414,9.194336,11.441620,8.976807,11.491058,6.790100,8.075775,8.022278,10.494049,10.908417,14.267517,12.395752,13.113617,14.213745,14.046021,13.631104,16.484131,10.706055,9.932129,11.448975,13.293854,11.003326,11.483887,0.000610,0.000473,0.000358,0.000413,0.000422,0.000538,0.000887,0.000573,0.000430,0.000813,0.000802,0.000385,6.0,48.0,0.0,0.0,0.0,0.0,25.0,66.0,17.0,0.0,19.0,42.0,0.0,99.0,83.0,0.0,22.0,39.0,0.0,0.0,0.0,10.0,32.0,11.0,0.0,0.0,1.0
4,1986,3.598450,12900.0,46420.0,historical,Ain,01,10.697601,11.505005,9.750580,11.271729,8.971222,12.694794,9.380066,10.372467,11.943848,12.838287,12.338928,9.982452,12.361023,13.699158,15.510254,15.075623,12.291718,16.009277,12.188660,14.697021,16.233276,10.914093,13.883301,13.264771,0.000214,0.000221,0.000266,0.000291,0.000588,0.000288,0.000724,0.000330,0.000420,0.000190,0.000690,0.000542,8.0,21.0,0.0,0.0,0.0,0.0,11.0,52.0,10.0,0.0,17.0,44.0,0.0,85.0,97.0,0.0,23

# Test of the module Feature Creation (feature_creation.py)


In [314]:
df_hist, df_forecast = FeatureCreation(df_yield, df_climate).run()

--- Amplitude feature created ---
--- rainy_days feature created ---
--- heat_days feature created ---
--- frost_days feature created ---


# Manual implementation of the feature creation module to test the functions


In [284]:
df_hist = df_yield
df_forecast = pd.DataFrame()

# Feature Creation


We will perform resample our df_climate dataset to set frequency $f$ that can be set to '1H' for hourly, '1D' for daily, '1W' for weekly, '1M' for monthly, '1Y' for yearly. We will then create new features based on the resampled data.


In [267]:
frequency = "1M"
group_col = ["scenario", "nom_dep", "code_dep", "year"]
metrics = df_climate.columns[~df_climate.columns.isin(group_col)]

## Amplitude


In [268]:
def amplitude(group):
    return group.max() - group.min()

In [283]:
monthly_data = (
    df_climate.groupby(by=group_col).resample(frequency).agg(amplitude).reset_index()
)
new_features = ["amp_" + feat for feat in metrics]
col_rename = {old_feat: new_feat for old_feat, new_feat in zip(metrics, new_features)}
monthly_data.rename(columns=col_rename, inplace=True)

In [285]:
monthly_data["date"] = monthly_data["date"].apply(lambda date: date.strftime("%b"))
monthly_data

,scenario,nom_dep,code_dep,year,date,amp_max_daily_NSA_temp,amp_daily_NSA_temp,amp_precipitation
0,historical,Ain,01,1982,Jan,9.683777,8.102631,0.000293
1,historical,Ain,01,1982,Feb,14.983551,12.605316,0.000305
2,historical,Ain,01,1982,Mar,19.126007,15.886414,0.000566
3,historical,Ain,01,1982,Apr,19.172760,14.585327,0.000333
4,historical,Ain,01,1982,May,12.611084,8.893372,0.000520
...,...,...,...,...,...,...,...,...
147127,ssp5_8_5,Yvelines,78,2050,Aug,10.849060,8.088684,0.000317
147128,ssp5_8_5,Yvelines,78,2050,Sep,16.233643,13.955444,0.000193
147129,ssp5_8_5,Yvelines,78,2050,Oct,8.712646,9.545502,0.000345
147130,ssp5_8_5,Yvelines,78,2050,Nov,11.965179,9.601196,0.000220


In [289]:
pivoted_df = monthly_data.pivot_table(
    index=["scenario", "nom_dep", "code_dep", "year"],
    columns="date",
    values=new_features,
).reset_index()
pivoted_df.columns = [
    f"{metric}_{month}".strip("_") for metric, month in pivoted_df.columns
]
pivoted_df.set_index(["year"], drop=True, inplace=True)
pivoted_df.index = pd.to_datetime(pivoted_df.index, format="%Y")
pivoted_df

,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,amp_daily_NSA_temp_Jun,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,historical,Ain,01,14.585327,13.266602,14.508118,12.605316,8.102631,8.545471,9.951599,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1983-01-01,historical,Ain,01,11.894348,7.150391,16.380951,17.021851,9.464539,10.034546,10.661621,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
1984-01-01,historical,Ain,01,14.128510,5.867737,12.289917,9.180328,15.847443,10.030670,9.929626,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
1985-01-01,historical,Ain,01,11.441620,8.075775,14.267517,13.399414,14.638794,6.790100,11.491058,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
1986-01-01,historical,Ain,01,11.271729,10.372467,9.982452,11.505005,10.697601,9.380066,12.694794,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,10.671600,11.138885,10.903961,10.098511,6.579498,10.647217,13.361725,...,0.000108,0.000067,0.000138,0.000076,0.000130,0.000076,0.000143,0.000382,0.000298,0.000022
2047-01-01,ssp5_8_5,Yvelines,78,5.617401,10.944977,10.930328,12.897888,16.145782,16.649200,14.308533,...,0.000135,0.000169,0.000239,0.000226,0.000100,0.000119,0.000227,0.000189,0.000077,0.000192
2048-01-01,ssp5_8_5,Yvelines,78,8.661102,8.688995,10.149933,13.638367,9.829681,13.737396,11.352478,...,0.000118,0.000153,0.000218,0.000495,0.000294,0.000374,0.000173,0.000214,0.000326,0.000256


Merge to df_yield on year and department.


In [290]:
df_hist = pd.merge(
    df_hist,
    pivoted_df[pivoted_df["scenario"] == "historical"],
    left_on=["department", "year"],
    right_on=["nom_dep", "year"],
    how="outer",
)

In [291]:
df_forecast = pivoted_df[~(pivoted_df["scenario"] == "historical")]

In [292]:
df_hist.head()

,department,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,Ain,3.950080,16065.0,63458.0,historical,Ain,01,14.585327,13.266602,14.508118,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1983-01-01,Ain,2.648276,14500.0,38400.0,historical,Ain,01,11.894348,7.150391,16.380951,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
1984-01-01,Ain,4.822580,15500.0,74750.0,historical,Ain,01,14.128510,5.867737,12.289917,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
1985-01-01,Ain,4.196770,15500.0,65050.0,historical,Ain,01,11.441620,8.075775,14.267517,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
1986-01-01,Ain,3.598450,12900.0,46420.0,historical,Ain,01,11.271729,10.372467,9.982452,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420


# Threshold Features


In [ ]:
thresholds = {"rain": 8.493e-05, "frost": 0 + 273.15, "heat": 30 + 273.15}


def rainy_days(group):
    return (group > thresholds["rain"]).sum()


def frost_days(group):
    return (group < thresholds["frost"]).sum()


def heat_days(group):
    return (group > thresholds["heat"]).sum()


rain_col = "precipitation"
max_temp_col = "max_daily_NSA_temp"
mean_temp_col = "daily_NSA_temp"

dic_agg = {rain_col: rainy_days, max_temp_col: heat_days, mean_temp_col: frost_days}

monthly_data = (
    df_climate.groupby(by=group_col).resample(frequency).agg(dic_agg).reset_index()
)

,scenario,nom_dep,code_dep,year,date,precipitation,max_daily_NSA_temp,daily_NSA_temp
0,historical,Ain,01,1982,1982-01-31,21,0,0
1,historical,Ain,01,1982,1982-02-28,8,0,10
2,historical,Ain,01,1982,1982-03-31,17,0,2
3,historical,Ain,01,1982,1982-04-30,13,0,0
4,historical,Ain,01,1982,1982-05-31,16,0,0
...,...,...,...,...,...,...,...,...
147127,ssp5_8_5,Yvelines,78,2050,2050-08-31,4,0,0
147128,ssp5_8_5,Yvelines,78,2050,2050-09-30,3,0,0
147129,ssp5_8_5,Yvelines,78,2050,2050-10-31,2,0,0
147130,ssp5_8_5,Yvelines,78,2050,2050-11-30,3,0,0


In [296]:
rename_dic = {col: func.__name__ for col, func in dic_agg.items()}
monthly_data.rename(columns=rename_dic, inplace=True)

monthly_data["date"] = monthly_data["date"].apply(lambda date: date.strftime("%b"))

monthly_data

,scenario,nom_dep,code_dep,year,date,rainy_days,heat_days,frost_days
0,historical,Ain,01,1982,Jan,21,0,0
1,historical,Ain,01,1982,Feb,8,0,10
2,historical,Ain,01,1982,Mar,17,0,2
3,historical,Ain,01,1982,Apr,13,0,0
4,historical,Ain,01,1982,May,16,0,0
...,...,...,...,...,...,...,...,...
147127,ssp5_8_5,Yvelines,78,2050,Aug,4,0,0
147128,ssp5_8_5,Yvelines,78,2050,Sep,3,0,0
147129,ssp5_8_5,Yvelines,78,2050,Oct,2,0,0
147130,ssp5_8_5,Yvelines,78,2050,Nov,3,0,0


In [305]:
# Pivot from monthly data to yearly data
pivoted_df = monthly_data.pivot_table(
    index=["scenario", "nom_dep", "code_dep", "year"],
    columns="date",
    values=list(rename_dic.values()),
).reset_index()

pivoted_df.columns = [
    f"{metric}_{month}".strip("_") for metric, month in pivoted_df.columns
]

In [306]:
pivoted_df.set_index(["year"], drop=True, inplace=True)
pivoted_df.index = pd.to_datetime(pivoted_df.index, format="%Y")
pivoted_df

,scenario,nom_dep,code_dep,frost_days_Apr,frost_days_Aug,frost_days_Dec,frost_days_Feb,frost_days_Jan,frost_days_Jul,frost_days_Jun,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,historical,Ain,01,0.0,0.0,15.0,10.0,0.0,0.0,0.0,...,7.0,8.0,21.0,3.0,2.0,17.0,16.0,6.0,5.0,2.0
1983-01-01,historical,Ain,01,0.0,0.0,21.0,11.0,11.0,0.0,0.0,...,7.0,11.0,14.0,6.0,8.0,13.0,10.0,18.0,4.0,9.0
1984-01-01,historical,Ain,01,1.0,0.0,15.0,10.0,19.0,0.0,0.0,...,8.0,8.0,14.0,7.0,11.0,7.0,13.0,11.0,11.0,5.0
1985-01-01,historical,Ain,01,1.0,0.0,15.0,19.0,13.0,0.0,0.0,...,9.0,13.0,9.0,12.0,5.0,5.0,18.0,8.0,17.0,2.0
1986-01-01,historical,Ain,01,0.0,0.0,8.0,2.0,11.0,0.0,0.0,...,17.0,2.0,7.0,7.0,3.0,10.0,10.0,8.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,2.0,0.0,3.0,0.0,1.0,0.0,3.0,5.0,3.0,0.0
2047-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,1.0,3.0,8.0,0.0,0.0,...,1.0,3.0,1.0,1.0,1.0,2.0,1.0,4.0,0.0,4.0
2048-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,3.0,9.0,6.0,0.0,0.0,...,2.0,5.0,2.0,2.0,4.0,2.0,2.0,6.0,3.0,2.0


In [307]:
pd.merge(
    df_hist,
    pivoted_df[pivoted_df["scenario"] == "historical"],
    left_on=["nom_dep", "year", "code_dep", "scenario"],
    right_on=["nom_dep", "year", "code_dep", "scenario"],
    how="outer",
)

,department,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,Ain,3.950080,16065.0,63458.0,historical,Ain,01,14.585327,13.266602,14.508118,...,7.0,8.0,21.0,3.0,2.0,17.0,16.0,6.0,5.0,2.0
1983-01-01,Ain,2.648276,14500.0,38400.0,historical,Ain,01,11.894348,7.150391,16.380951,...,7.0,11.0,14.0,6.0,8.0,13.0,10.0,18.0,4.0,9.0
1984-01-01,Ain,4.822580,15500.0,74750.0,historical,Ain,01,14.128510,5.867737,12.289917,...,8.0,8.0,14.0,7.0,11.0,7.0,13.0,11.0,11.0,5.0
1985-01-01,Ain,4.196770,15500.0,65050.0,historical,Ain,01,11.441620,8.075775,14.267517,...,9.0,13.0,9.0,12.0,5.0,5.0,18.0,8.0,17.0,2.0
1986-01-01,Ain,3.598450,12900.0,46420.0,historical,Ain,01,11.271729,10.372467,9.982452,...,17.0,2.0,7.0,7.0,3.0,10.0,10.0,8.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-01,Yvelines,7.037840,9624.0,67732.2,historical,Yvelines,78,9.208282,10.101349,10.645935,...,2.0,2.0,3.0,4.0,0.0,1.0,1.0,5.0,2.0,1.0
2011-01-01,Yvelines,6.155870,8766.0,53962.4,historical,Yvelines,78,9.366638,13.055542,8.522217,...,3.0,1.0,1.0,1.0,4.0,5.0,8.0,3.0,6.0,1.0
2012-01-01,Yvelines,7.675490,9100.0,69847.0,historical,Yvelines,78,11.627319,11.498718,12.754639,...,2.0,1.0,2.0,0.0,0.0,2.0,3.0,5.0,5.0,2.0


In [309]:
pd.merge(
    df_forecast,
    pivoted_df[~(pivoted_df["scenario"] == "historical")],
    on=["nom_dep", "year", "scenario", "code_dep"],
    how="outer",
)

,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,amp_daily_NSA_temp_Jun,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
2015-01-01,ssp1_2_6,Ain,01,13.302734,10.143005,15.577698,6.312073,5.736542,11.332397,9.489990,...,7.0,8.0,10.0,6.0,3.0,12.0,8.0,9.0,4.0,11.0
2016-01-01,ssp1_2_6,Ain,01,12.063354,11.057465,9.628601,11.505615,8.315643,8.163452,8.313171,...,9.0,9.0,12.0,11.0,5.0,5.0,13.0,14.0,15.0,6.0
2017-01-01,ssp1_2_6,Ain,01,8.806732,10.507019,12.481995,11.021942,13.921326,6.458191,10.793091,...,11.0,9.0,11.0,6.0,5.0,8.0,6.0,12.0,10.0,5.0
2018-01-01,ssp1_2_6,Ain,01,14.717407,8.809509,13.714783,6.127136,10.296631,11.391785,8.336487,...,11.0,7.0,9.0,8.0,8.0,7.0,18.0,18.0,17.0,6.0
2019-01-01,ssp1_2_6,Ain,01,16.814819,8.586914,9.677094,12.183472,8.142578,14.037292,10.136383,...,11.0,5.0,6.0,9.0,11.0,5.0,8.0,15.0,13.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,10.671600,11.138885,10.903961,10.098511,6.579498,10.647217,13.361725,...,2.0,0.0,3.0,0.0,1.0,0.0,3.0,5.0,3.0,0.0
2047-01-01,ssp5_8_5,Yvelines,78,5.617401,10.944977,10.930328,12.897888,16.145782,16.649200,14.308533,...,1.0,3.0,1.0,1.0,1.0,2.0,1.0,4.0,0.0,4.0
2048-01-01,ssp5_8_5,Yvelines,78,8.661102,8.688995,10.149933,13.638367,9.829681,13.737396,11.352478,...,2.0,5.0,2.0,2.0,4.0,2.0,2.0,6.0,3.0,2.0
